### import statements

In [ ]:
%matplotlib inline
import sys
import os
import pandas as pd
sys.path.append('../src')
import datetime
import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from plot_utils import *

### constants

In [ ]:
F_INPUT = '../../data/v2/data.h5'

### read data

In [ ]:
df = pd.read_hdf(F_INPUT, 'merged')

### add new features

In [ ]:
df['day'] = df.index.dayofyear
df['logCn2'] = np.log10(df['Cn2'])

In [ ]:
df['r0_2T'] = df['r0'].rolling('2T').mean()
df['r0_day_2T'] = df['r0_day'].rolling('2T').mean()
df['r0_night_2T'] = df['r0_night'].rolling('2T').mean()

In [ ]:
df['r0_3T'] = df['r0'].rolling('3T').mean()
df['r0_day_3T'] = df['r0_day'].rolling('3T').mean()
df['r0_night_3T'] = df['r0_night'].rolling('3T').mean()

In [ ]:
df['r0_5T'] = df['r0'].rolling('5T').mean()
df['r0_day_5T'] = df['r0_day'].rolling('5T').mean()
df['r0_night_5T'] = df['r0_night'].rolling('5T').mean()

In [ ]:
df['r0_10T'] = df['r0'].rolling('10T').mean()
df['r0_day_10T'] = df['r0_day'].rolling('10T').mean()
df['r0_night_10T'] = df['r0_night'].rolling('10T').mean()

In [ ]:
df['r0_15T'] = df['r0'].rolling('15T').mean()
df['r0_day_15T'] = df['r0_day'].rolling('15T').mean()
df['r0_night_15T'] = df['r0_night'].rolling('15T').mean()

In [ ]:
df['r0_20T'] = df['r0'].rolling('20T').mean()
df['r0_day_20T'] = df['r0_day'].rolling('20T').mean()
df['r0_night_20T'] = df['r0_night'].rolling('20T').mean()

In [ ]:
feats = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'logCn2', 'solar_zenith_angle','day']
feats_minus_cn2 = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'solar_zenith_angle','day']

label_day = 'r0_day'
label = 'r0'
label_night = 'r0_night'
feats_plus_r0 = feats + ['r0']
feats_plus_r0day = feats + ['r0_day']
feats_plus_r0night = feats + ['r0_night']

In [ ]:
xmin, ymin = 0, 0
xmax, ymax = 20, 20

### restricting data to usable, relatively dense subset

In [ ]:
df_subset = df[(df.index > '2018-05-03') & (df.index < '2020-12-30')]

In [ ]:
df_subset.describe()

### finding non-nan values

In [ ]:
valid = ~df_subset[feats_plus_r0].isnull().any(axis=1)

In [ ]:
df_subset.loc[valid, feats_plus_r0].count()

In [ ]:
valid_day = ~df_subset[feats_plus_r0day].isnull().any(axis=1)

In [ ]:
df_subset.loc[valid_day, feats_plus_r0day].count()

In [ ]:
valid_night = ~df_subset[feats_plus_r0night].isnull().any(axis=1)

In [ ]:
df_subset.loc[valid_night,feats_plus_r0night].count()

In [ ]:
# df_subset.loc[valid_night,feats + ['r0_night_20T']].count()
# df_subset.loc[valid_day,feats + ['r0_day_20T']].count()
# df_subset.loc[valid,feats + ['r0_20T']].count()

### splitting into train and test

In [ ]:
split_date = '2019-12-31'
train = df_subset.index <= split_date
test  = df_subset.index > split_date

In [ ]:
test_truth_night = df_subset.loc[test&valid_night,label_night]
test_truth_day = df_subset.loc[test&valid_day,label_day]
test_truth_all = df_subset.loc[test&valid,label]

In [ ]:
df_subset.loc[train&valid,feats_plus_r0].count()

In [ ]:
df_subset.loc[test&valid,feats_plus_r0].count()

In [ ]:
df_subset.loc[train&valid_day,feats_plus_r0day].count()

In [ ]:
df_subset.loc[test&valid_day,feats_plus_r0day].count()

In [ ]:
df_subset.loc[train&valid_night,feats_plus_r0night].count()

In [ ]:
df_subset.loc[test&valid_night,feats_plus_r0night].count()

### initializing the RF regressor

In [ ]:
regr = RandomForestRegressor(n_estimators=100, random_state=0)

### train and test subroutine

In [ ]:
def train_and_test(train_df, test_df, feats, label):
    forest = regr.fit(train_df[feats], train_df[label])
#     r2 = regr.score(test_df[feats], test_df[label])
    preds = regr.predict(test_df[feats])
    r2 = r2_score(test_df[label], preds)
    sq_err = mean_squared_error(test_df[label], preds)
    perc_err = mean_absolute_percentage_error(test_df[label], preds)
    return {'forest': forest, 'preds': preds, 'r2': r2, 'sq_err': sq_err, 'perc_err': perc_err}

### Get All Results

In [ ]:
results_all_1T = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, label)

In [ ]:
# scatter_with_errors(test_truth, test_preds_day, test_perc_err_day, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)

In [ ]:
results_day_1T = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, label_day)

In [ ]:
# scatter_with_errors(test_truth_day, test_preds_day, test_perc_err_day, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)

In [ ]:
results_night_1T = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, label_night)

In [ ]:
results_all_2T = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, 'r0_2T')

In [ ]:
results_day_2T = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, 'r0_day_2T')

In [ ]:
results_night_2T = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, 'r0_night_2T')

In [ ]:
results_all_5T = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, 'r0_5T')

In [ ]:
results_day_5T = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, 'r0_day_5T')

In [ ]:
results_night_5T = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, 'r0_night_5T')

In [ ]:
results_all_10T = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, 'r0_10T')

In [ ]:
results_day_10T = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, 'r0_day_10T')

In [ ]:
results_night_10T = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, 'r0_night_10T')

In [ ]:
results_all_15T = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, 'r0_15T')

In [ ]:
results_day_15T = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, 'r0_day_15T')

In [ ]:
results_night_15T = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, 'r0_night_15T')

In [ ]:
results_all_20T = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, 'r0_20T')

In [ ]:
results_day_20T = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, 'r0_day_20T')

In [ ]:
results_night_20T = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, 'r0_night_20T')

### Compare all Results

In [ ]:
for r in [ results_all_1T, results_all_2T, results_all_5T, results_all_10T, results_all_15T, results_all_20T ]:
    print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
for r in [ results_day_1T, results_day_2T, results_day_5T, results_day_10T, results_day_15T, results_day_20T ]:
    print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
for r in [ results_night_1T, results_night_2T, results_night_5T, results_night_10T, results_night_15T, results_night_20T ]:
    print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# scatter_with_errors(test_truth_day, test_preds_day, test_perc_err_day, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)

In [ ]:
# scatter_with_errors(test_truth_day, test_preds_day, test_perc_err_day, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)

In [ ]:
# error_by_r0_histograms(test_truth_all, error_perc(test_truth_all, test_preds_all), 0, 80)
# error_by_r0_histograms(test_truth_day, error_perc(test_truth_day, test_preds_day), 0, 80)
# error_by_r0_histograms(test_truth_night, error_perc(test_truth_night, test_preds_night), 0, 80)

In [ ]:
test_truth_night = df_subset.loc[test&valid_night,label_night]
test_truth_day = df_subset.loc[test&valid_day,label_day]
test_truth_all = df_subset.loc[test&valid,label]

In [ ]:
test_truth_night_15T = df_subset.loc[test&valid_night,'r0_night_15T']
test_pred_night_15T = results_night_10T['preds']
error_by_r0_histograms(test_truth_night_15T, error_perc(test_truth_night_15T, test_pred_night_15T), 0, 80)
scatter_with_errors(test_truth_night_15T, test_pred_night_15T, error_perc, xmin, xmax, ymin, ymax)

In [ ]:
test_truth_day_15T = df_subset.loc[test&valid_day,'r0_day_15T']
test_pred_day_15T = results_day_15T['preds']
error_by_r0_histograms(test_truth_day_15T, error_perc(test_truth_day_15T, test_pred_day_15T), 0, 90)
scatter_with_errors(test_truth_day_15T, test_pred_day_15T, error_perc, xmin, xmax, ymin, ymax)

### Plotting

#### scatter plots of actual vs. predict using error_diff

In [ ]:
# %matplotlib inline
# scatter_with_errors(test_truth_all, test_preds_all, error_perc, xmin, xmax, ymin, ymax)

# scatter_with_errors(test_truth_night, test_preds_night, error_perc, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_night,feats], feats)

# scatter_with_errors(test_truth_night, test_preds_night, error_perc, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_night,feats], feats)

#### interactive time domain plot of errors

switching matplotlib to notebook mode to enable a zoom-in of different portions of the time axis

In [ ]:
# %matplotlib notebook 
# plot_errors_in_time(test_truth_all, test_preds_all)

#### feature importance

feature importance from the model

In [ ]:
# %matplotlib inline

In [ ]:
# plot_importance(regr, df_subset.loc[train&valid,feats], feats)

## debug why CN2 is so low

#### What happens if I drop month and SZA

Answer: turns out we had to take the log of CN2

In [ ]:
# feats_no_sza = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'logCn2']
# preds_all_no_sza, r2_all_no_sza = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats_no_sza, label)
# scatter_with_errors(test_truth_all, preds_all_no_sza, error_perc, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid,feats_no_sza], feats_no_sza)
# r2_all_no_sza

#### correlation between the signals using [stats.pearsonr](https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9)

We calculate:
- overall synchrony between r0 and Cn2
- local synchrony between r0 and Cn2

#### Overall Synchrony

In [ ]:
    # r, p = print_pearsonr(df_subset.loc[train&valid,label], df_subset.loc[train&valid,'logCn2'])
# plot_overall_synchrony(df_subset.loc[train&valid,label], df_subset.loc[train&valid,['logCn2']], label, 'logCn2', r)

#### Local Synchrony

In [ ]:
# plot_local_synchrony(df_subset.loc[train&valid,label], df_subset.loc[train&valid,['logCn2']], label, 'Cn2')

## Synchrony using only R0 daytime data

#### Overall Synchrony

In [ ]:
# r, p = print_pearsonr(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,'logCn2'])
# r

In [ ]:
# plot_overall_synchrony(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,['logCn2']], label_day, 'logCn2', r)

#### Local Synchrony

In [ ]:
# plot_local_synchrony(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,['logCn2']], label_day, 'logCn2')

## Try comparing results without CN2

In [ ]:
results_all_15T_nocn2 = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats_minus_cn2, 'r0_15T')

In [ ]:
results_day_15T_nocn2 = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats_minus_cn2, 'r0_day_15T')

In [ ]:
results_night_15T_nocn2 = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats_minus_cn2, 'r0_night_15T')

In [ ]:
for r in [ results_all_15T, results_all_15T_nocn2 ]:
    print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
for r in [ results_day_15T, results_day_15T_nocn2 ]:
    print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
for r in [ results_night_15T, results_night_15T_nocn2 ]:
    print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
plot_importance(results_all_15T['forest'], df_subset.loc[train & valid, feats], feats)

In [ ]:
# test_truth_night_15T = df_subset.loc[test&valid_night,'r0_night_15T']
# test_pred_night_15T = results_night_10T['preds']
# error_by_r0_histograms(test_truth_night_15T, error_perc(test_truth_night_15T, test_pred_night_15T), 0, 80)
# scatter_with_errors(test_truth_night_15T, test_pred_night_15T, error_perc, xmin, xmax, ymin, ymax)

## Why R2 so weird?

In [ ]:
print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night'])))
print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_2T'])))
print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_5T'])))
print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_10T'])))
print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_15T'])))
print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_20T'])))

## Plot One Day

In [ ]:
def plot_one_day(preds, smooth_df, orig_df, startdate, enddate):

    preds_df = pd.DataFrame(preds, index=smooth_df.index) 

    daymask = (orig_df.index > startdate ) & (orig_df.index < enddate)

    plt.figure(figsize=(20, 5))
    plt.plot(orig_df[daymask], 'g.', label='r0')
    plt.plot(smooth_df[daymask], 'b.', label='r0 smoothed (15min)')
    plt.plot(preds_df[daymask], 'r.', label='preds')
    plt.ylabel('r0')
    plt.legend()
    return

In [ ]:
plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-05-30 04:00', '2020-05-31')


In [ ]:
plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-08-30 04:00', '2020-08-31 00:00')

In [ ]:
plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-02-15 04:00', '2020-02-16 00:00')

In [ ]:
plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-11-15 04:00', '2020-11-16 00:00')